In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


Train model with explanation + repair with 80-10-10 dataset
5:48 start 6:01 end

In [6]:
import pandas as pd
import numpy as np
from transformers import RobertaTokenizer, T5ForConditionalGeneration, T5Config, AutoModelForSeq2SeqLM, AutoTokenizer, get_linear_schedule_with_warmup
from torch.utils.data import DataLoader, Dataset, RandomSampler, SequentialSampler
import torch
import torch.optim as optim
from tqdm import tqdm
import logging
import os
import spacy
from nltk.translate.bleu_score import corpus_bleu, SmoothingFunction, sentence_bleu
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import TfidfVectorizer
import textstat


logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

class Args:
    def __init__(self):
        self.train_data_file = '/content/drive/MyDrive/9900training/801010dataset/train.csv'
        self.eval_data_file = '/content/drive/MyDrive/9900training/801010dataset/val.csv'
        self.test_data_file = '/content/drive/MyDrive/9900training/801010dataset/test.csv'
        self.block_size = 512
        self.train_batch_size = 4
        self.val_batch_size = 4
        self.epochs = 10
        # # # # quick test
        # self.epochs = 1
        # self.sample_size = 50

args = Args()

model = AutoModelForSeq2SeqLM.from_pretrained('Salesforce/codet5-base-multi-sum')
tokenizer = AutoTokenizer.from_pretrained('Salesforce/codet5-base-multi-sum')

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)


class InputFeatures(object):
    def __init__(self, input_tokens, input_ids, descs):
        self.input_tokens = input_tokens
        self.input_ids = input_ids
        self.descs = descs

def convert_examples_to_features(func, desc, tokenizer, args):
    source_tokens = tokenizer.tokenize(str(func))[:args.block_size-2]
    # deleted that part because we dont know the details of t5 tokenizer, that
    # is particularlly for codebert
    source_ids = tokenizer.convert_tokens_to_ids(source_tokens)
    padding_length = args.block_size - len(source_ids)
    source_ids += [tokenizer.pad_token_id] * padding_length
    return InputFeatures(source_tokens, source_ids, str(desc))

class CodeToTextDataset(Dataset):
    def __init__(self, tokenizer, args, file_type):
        if file_type == "train":
            file_path = args.train_data_file
        elif file_type == "eval":
            file_path = args.eval_data_file
        elif file_type == "test":
            file_path = args.test_data_file

        self.examples = []
        df = pd.read_csv(file_path)
        # quick testing
        # df = pd.read_csv(file_path).sample(n=args.sample_size)
        funcs = df["vulnerable_code"].tolist()

        descs_explain = df["Explanation_of_Vulnerability_In_Context"].tolist()
        descs_change = df["Explanation_Vulnerability_Fixed_In_Context"].tolist()

        descs = []
        for i in range(len(descs_explain)):
            descs.append(str(descs_explain[i]) + str(descs_change[i]))

        for i in tqdm(range(len(funcs)), desc="Loading data"):
            self.examples.append(convert_examples_to_features(funcs[i],descs[i], tokenizer, args))
        if file_type == "train":
            for example in self.examples[:3]:
                logger.info("*** Example ***")
                logger.info("desc: {}".format(example.descs))
                logger.info("input_tokens: {}".format([x.replace('\u0120','_') for x in example.input_tokens]))
                logger.info("input_ids: {}".format(' '.join(map(str, example.input_ids))))

    def __len__(self):
        return len(self.examples)

    def __getitem__(self, idx):
        example = self.examples[idx]
        return {
            'input_ids': torch.tensor(example.input_ids, dtype=torch.long),
            'desc': torch.tensor(tokenizer.encode(example.descs, padding='max_length', truncation=True, max_length=args.block_size), dtype=torch.long),
        }

def train(train_dataset, model, args, tokenizer):
    train_sampler = RandomSampler(train_dataset)
    train_dataloader = DataLoader(train_dataset, sampler=train_sampler, batch_size=args.train_batch_size, num_workers=0)

    optimizer = optim.AdamW(model.parameters(), lr=5e-5)
    total_steps = len(train_dataloader) * args.epochs
    scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=0, num_training_steps=total_steps)

    for epoch in range(args.epochs):
        model.train()
        for batch in train_dataloader:
            input_ids = batch['input_ids'].to(device)
            descs = batch['desc'].to(device)

            outputs = model(input_ids=input_ids, labels=descs)
            loss = outputs.loss

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            scheduler.step()

            print(f'Epoch: {epoch}, Loss: {loss.item()}')
    model_to_save = model.module if hasattr(model,'module') else model
    torch.save(model_to_save.state_dict(), '/content/drive/MyDrive/9900training/models/801010_description_repair_train_t5.bin')
    # torch.save(model_to_save.state_dict(), '/content/drive/MyDrive/9900training/custom_train_t5.bin')


def test(test_dataset, model, tokenizer, args):
    test_sampler = SequentialSampler(test_dataset)
    test_dataloader = DataLoader(test_dataset, sampler=test_sampler, batch_size=1, num_workers=0)

    model.eval()
    total_loss = 0
    for batch in tqdm(test_dataloader, desc="Testing"):
        input_ids = batch['input_ids'].to(device)
        true_desc = batch['desc'].to(device)

        with torch.no_grad():
            outputs = model.generate(input_ids=input_ids, max_length=args.block_size)

        predicted_desc = tokenizer.decode(outputs[0], skip_special_tokens=True)
        true_desc_decoded = tokenizer.decode(true_desc[0], skip_special_tokens=True)

        logger.info(f"True Description: {true_desc_decoded}")
        logger.info(f"Predicted Description: {predicted_desc}")
        print(f"True Description: {true_desc_decoded}")
        print(f"Predicted Description: {predicted_desc}")


def test_model(test_dataset, tokenizer, args):
    path_to_model = '/content/drive/MyDrive/9900training/models/801010_description_repair_train_t5.bin'
    model.load_state_dict(torch.load(path_to_model))
    test_sampler = SequentialSampler(test_dataset)
    test_dataloader = DataLoader(test_dataset, sampler=test_sampler, batch_size=1, num_workers=0)

    model.eval()
    total_loss = 0
    for batch in tqdm(test_dataloader, desc="Testing"):
        input_ids = batch['input_ids'].to(device)
        true_desc = batch['desc'].to(device)

        with torch.no_grad():
            outputs = model.generate(input_ids=input_ids, max_length=args.block_size)

        predicted_desc = tokenizer.decode(outputs[0], skip_special_tokens=True)
        true_desc_decoded = tokenizer.decode(true_desc[0], skip_special_tokens=True)

        logger.info(f"True Description: {true_desc_decoded}")
        logger.info(f"Predicted Description: {predicted_desc}")
        print(f"True Description: {true_desc_decoded}")
        print(f"Predicted Description: {predicted_desc}")

def evaluate_model(val_dataset, model, args, tokenizer):
    path_to_model = '/content/drive/MyDrive/9900training/models/801010_description_repair_train_t5.bin'
    model.load_state_dict(torch.load(path_to_model))
    model.to(device)

    val_sampler = SequentialSampler(val_dataset)
    val_dataloader = DataLoader(val_dataset, sampler=val_sampler, batch_size=1, num_workers=0)

    model.eval()
    total_loss = 0
    bleu_scores = []
    cosine_similarities = []
    flesch_kincaid_scores = []

    tfidf_vectorizer = TfidfVectorizer()

    for batch in tqdm(val_dataloader, desc="Evaluating"):
        input_ids = batch['input_ids'].to(device)
        descs = batch['desc'].to(device)

        with torch.no_grad():
            outputs = model(input_ids=input_ids, labels=descs)
            loss = outputs.loss
            total_loss += loss.item()

            predicted_desc = tokenizer.decode(outputs.logits.argmax(-1)[0], skip_special_tokens=True)
            true_desc = tokenizer.decode(descs[0], skip_special_tokens=True)

            # BLEU-4 score
            smoothie = SmoothingFunction().method4
            bleu_score = sentence_bleu([true_desc.split()], predicted_desc.split(), smoothing_function=smoothie, weights=(0.25, 0.25, 0.25, 0.25))
            bleu_scores.append(bleu_score)

            # Cosine similarity
            tfidf_matrix = tfidf_vectorizer.fit_transform([true_desc, predicted_desc])
            cosine_sim = cosine_similarity(tfidf_matrix[0:1], tfidf_matrix[1:2])
            cosine_similarities.append(cosine_sim[0][0])

            # Flesch-Kincaid Grade Level
            flesch_kincaid_true = textstat.flesch_kincaid_grade(true_desc)
            flesch_kincaid_pred = textstat.flesch_kincaid_grade(predicted_desc)
            flesch_kincaid_diff = abs(flesch_kincaid_true - flesch_kincaid_pred)
            flesch_kincaid_scores.append(flesch_kincaid_diff)

    avg_bleu = np.mean(bleu_scores)
    avg_cosine_sim = np.mean(cosine_similarities)
    avg_flesch_kincaid = np.mean(flesch_kincaid_scores)

    print(f'Validation Loss: {total_loss / len(val_dataloader)}')
    print(f'Average BLEU-4 Score: {avg_bleu}')
    print(f'Average Cosine Similarity: {avg_cosine_sim}')
    print(f'Average Flesch-Kincaid Grade Level Difference: {avg_flesch_kincaid}')

def evaluate(val_dataset, model, args, tokenizer):
    val_sampler = SequentialSampler(val_dataset)
    val_dataloader = DataLoader(val_dataset, sampler=val_sampler, batch_size=args.val_batch_size, num_workers=0)

    model.eval()
    total_loss = 0
    bleu_scores = []
    cosine_similarities = []
    flesch_kincaid_scores = []

    tfidf_vectorizer = TfidfVectorizer()

    for batch in tqdm(val_dataloader, desc="Evaluating"):
        input_ids = batch['input_ids'].to(device)
        descs = batch['desc'].to(device)

        with torch.no_grad():
            outputs = model(input_ids=input_ids, labels=descs)
            loss = outputs.loss
            total_loss += loss.item()

            predicted_desc = tokenizer.decode(outputs.logits.argmax(-1)[0], skip_special_tokens=True)
            true_desc = tokenizer.decode(descs[0], skip_special_tokens=True)

            # BLEU-4 score
            smoothie = SmoothingFunction().method4
            bleu_score = sentence_bleu([true_desc.split()], predicted_desc.split(), smoothing_function=smoothie, weights=(0.25, 0.25, 0.25, 0.25))
            bleu_scores.append(bleu_score)

            # Cosine similarity
            tfidf_matrix = tfidf_vectorizer.fit_transform([true_desc, predicted_desc])
            cosine_sim = cosine_similarity(tfidf_matrix[0:1], tfidf_matrix[1:2])
            cosine_similarities.append(cosine_sim[0][0])

            # Flesch-Kincaid Grade Level
            flesch_kincaid_true = textstat.flesch_kincaid_grade(true_desc)
            flesch_kincaid_pred = textstat.flesch_kincaid_grade(predicted_desc)
            flesch_kincaid_diff = abs(flesch_kincaid_true - flesch_kincaid_pred)
            flesch_kincaid_scores.append(flesch_kincaid_diff)

    avg_bleu = np.mean(bleu_scores)
    avg_cosine_sim = np.mean(cosine_similarities)
    avg_flesch_kincaid = np.mean(flesch_kincaid_scores)

    print(f'Validation Loss: {total_loss / len(val_dataloader)}')
    print(f'Average BLEU-4 Score: {avg_bleu}')
    print(f'Average Cosine Similarity: {avg_cosine_sim}')
    print(f'Average Flesch-Kincaid Grade Level Difference: {avg_flesch_kincaid}')


def generate_text(model, tokenizer, code, max_len=1024):
    model.eval()
    input_enc = tokenizer(code, max_length=max_len, padding='max_length', truncation=True, return_tensors="pt")
    input_ids = input_enc['input_ids'].to(device)

    with torch.no_grad():
        outputs = model.generate(input_ids=input_ids, max_length=max_len)

    return tokenizer.decode(outputs[0], skip_special_tokens=True)

train_dataset = CodeToTextDataset(tokenizer, args, 'train')
train(train_dataset, model, args, tokenizer)

test_dataset = CodeToTextDataset(tokenizer, args, 'test')
test(test_dataset, model, tokenizer, args)

val_dataset = CodeToTextDataset(tokenizer, args, 'eval')
evaluate(val_dataset, model, args, tokenizer)

# test_dataset = CodeToTextDataset(tokenizer, args, 'test')
# test_model(test_dataset, tokenizer, args)

# val_dataset = CodeToTextDataset(tokenizer, args, 'eval')
# evaluate_model(val_dataset, model, args, tokenizer)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/902 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/892M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.48k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/703k [00:00<?, ?B/s]

...

True Description: The vulnerability in the `cs_winkernel_malloc` function was caused by an unchecked addition of the size of the memory block structure to the requested size, potentially leading to an integer overflow. This unchecked arithmetic operation could result in a value that exceeds the capacity of the data type, causing a heap-based buffer overflow in a kernel driver. Attackers could exploit this vulnerability by providing a large value for memory allocation, triggering the overflow and potentially causing a denial of service or other unspecified impacts due to the buffer overflow in the kernel driver.The vulnerability was fixed by introducing a check using the `RtlSizeTAdd` function to calculate the total number of bytes to allocate, ensuring that the addition of the size of the memory block structure and the requested size does not result in an overflow or underflow. This function performs the addition in a safe manner, preventing arithmetic errors that could lead to memory 

Testing: 100%|██████████| 152/152 [10:47<00:00,  4.26s/it]

True Description: The vulnerability in the `verify_signed_hash` function was caused by the lack of validation for the padding contents within the signature during PKCS#1 v1.5 signature verification. This absence of verification allowed an attacker to manipulate the padding string, potentially leading to the forging of signatures, especially when small public exponents were used. The vulnerability affected the IKEv2 signature verification process when RAW RSA keys were utilized, enabling remote attackers to exploit the weakness in the RSA implementation and create fraudulent signatures by tampering with the padding values.In fixing the vulnerability, the code was updated to include a new functionality that verifies the padding contents within the signature after skipping the padding. By introducing a check to validate the padding contents against the expected pattern of byte values, the code now ensures the integrity of the padding content, preventing potential manipulation by attackers


Evaluating: 100%|██████████| 28/28 [00:02<00:00, 10.36it/s]

Validation Loss: 0.8225642421415874
Average BLEU-4 Score: 0.19617982899356876
Average Cosine Similarity: 0.7054646266252316
Average Flesch-Kincaid Grade Level Difference: 2.135714285714286


vul des + change -> model trained with original equal split database
7:04 start 7:40 ends

In [2]:
import pandas as pd
import numpy as np
from transformers import RobertaTokenizer, T5ForConditionalGeneration, T5Config, AutoModelForSeq2SeqLM, AutoTokenizer, get_linear_schedule_with_warmup
from torch.utils.data import DataLoader, Dataset, RandomSampler, SequentialSampler
import torch
import torch.optim as optim
from tqdm import tqdm
import logging
import os
import spacy


logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

class Args:
    def __init__(self):
        self.train_data_file = '/content/drive/MyDrive/9900training/train.csv'
        self.eval_data_file = '/content/drive/MyDrive/9900training/val.csv'
        self.test_data_file = '/content/drive/MyDrive/9900training/test.csv'
        self.block_size = 512
        self.train_batch_size = 4
        self.val_batch_size = 4
        self.epochs = 10
        # # # # quick test
        # self.epochs = 1
        # self.sample_size = 50

args = Args()

model = AutoModelForSeq2SeqLM.from_pretrained('Salesforce/codet5-base-multi-sum')
tokenizer = AutoTokenizer.from_pretrained('Salesforce/codet5-base-multi-sum')

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)


class InputFeatures(object):
    def __init__(self, input_tokens, input_ids, descs):
        self.input_tokens = input_tokens
        self.input_ids = input_ids
        self.descs = descs

def convert_examples_to_features(func, desc, tokenizer, args):
    source_tokens = tokenizer.tokenize(str(func))[:args.block_size-2]
    # deleted that part because we dont know the details of t5 tokenizer, that
    # is particularlly for codebert
    source_ids = tokenizer.convert_tokens_to_ids(source_tokens)
    padding_length = args.block_size - len(source_ids)
    source_ids += [tokenizer.pad_token_id] * padding_length
    return InputFeatures(source_tokens, source_ids, str(desc))

class CodeToTextDataset(Dataset):
    def __init__(self, tokenizer, args, file_type):
        if file_type == "train":
            file_path = args.train_data_file
        elif file_type == "eval":
            file_path = args.eval_data_file
        elif file_type == "test":
            file_path = args.test_data_file

        self.examples = []
        df = pd.read_csv(file_path)
        # quick testing
        # df = pd.read_csv(file_path).sample(n=args.sample_size)
        funcs = df["vulnerable_code"].tolist()

        descs_explain = df["Explanation_of_Vulnerability_In_Context"].tolist()
        descs_change = df["Explanation_Vulnerability_Fixed_In_Context"].tolist()

        descs = []
        for i in range(len(descs_explain)):
            descs.append(str(descs_explain[i]) + str(descs_change[i]))

        for i in tqdm(range(len(funcs)), desc="Loading data"):
            self.examples.append(convert_examples_to_features(funcs[i],descs[i], tokenizer, args))
        if file_type == "train":
            for example in self.examples[:3]:
                logger.info("*** Example ***")
                logger.info("desc: {}".format(example.descs))
                logger.info("input_tokens: {}".format([x.replace('\u0120','_') for x in example.input_tokens]))
                logger.info("input_ids: {}".format(' '.join(map(str, example.input_ids))))

    def __len__(self):
        return len(self.examples)

    def __getitem__(self, idx):
        example = self.examples[idx]
        return {
            'input_ids': torch.tensor(example.input_ids, dtype=torch.long),
            'desc': torch.tensor(tokenizer.encode(example.descs, padding='max_length', truncation=True, max_length=args.block_size), dtype=torch.long),
        }

def train(train_dataset, model, args, tokenizer):
    train_sampler = RandomSampler(train_dataset)
    train_dataloader = DataLoader(train_dataset, sampler=train_sampler, batch_size=args.train_batch_size, num_workers=0)

    optimizer = optim.AdamW(model.parameters(), lr=5e-5)
    total_steps = len(train_dataloader) * args.epochs
    scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=0, num_training_steps=total_steps)

    for epoch in range(args.epochs):
        model.train()
        for batch in train_dataloader:
            input_ids = batch['input_ids'].to(device)
            descs = batch['desc'].to(device)

            outputs = model(input_ids=input_ids, labels=descs)
            loss = outputs.loss

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            scheduler.step()

            print(f'Epoch: {epoch}, Loss: {loss.item()}')
    model_to_save = model.module if hasattr(model,'module') else model
    torch.save(model_to_save.state_dict(), '/content/drive/MyDrive/9900training/second_description_repair_train_t5.bin')
    # torch.save(model_to_save.state_dict(), '/content/drive/MyDrive/9900training/custom_train_t5.bin')


def test(test_dataset, model, tokenizer, args):
    test_sampler = SequentialSampler(test_dataset)
    test_dataloader = DataLoader(test_dataset, sampler=test_sampler, batch_size=1, num_workers=0)

    model.eval()
    total_loss = 0
    for batch in tqdm(test_dataloader, desc="Testing"):
        input_ids = batch['input_ids'].to(device)
        true_desc = batch['desc'].to(device)

        with torch.no_grad():
            outputs = model.generate(input_ids=input_ids, max_length=args.block_size)

        predicted_desc = tokenizer.decode(outputs[0], skip_special_tokens=True)
        true_desc_decoded = tokenizer.decode(true_desc[0], skip_special_tokens=True)

        logger.info(f"True Description: {true_desc_decoded}")
        logger.info(f"Predicted Description: {predicted_desc}")
        print(f"True Description: {true_desc_decoded}")
        print(f"Predicted Description: {predicted_desc}")


def test_model(test_dataset, tokenizer, args):
    path_to_model = '/content/drive/MyDrive/9900training/second_description_repair_train_t5.bin'
    model.load_state_dict(torch.load(path_to_model))
    test_sampler = SequentialSampler(test_dataset)
    test_dataloader = DataLoader(test_dataset, sampler=test_sampler, batch_size=1, num_workers=0)

    model.eval()
    total_loss = 0
    for batch in tqdm(test_dataloader, desc="Testing"):
        input_ids = batch['input_ids'].to(device)
        true_desc = batch['desc'].to(device)

        with torch.no_grad():
            outputs = model.generate(input_ids=input_ids, max_length=args.block_size)

        predicted_desc = tokenizer.decode(outputs[0], skip_special_tokens=True)
        true_desc_decoded = tokenizer.decode(true_desc[0], skip_special_tokens=True)

        logger.info(f"True Description: {true_desc_decoded}")
        logger.info(f"Predicted Description: {predicted_desc}")
        print(f"True Description: {true_desc_decoded}")
        print(f"Predicted Description: {predicted_desc}")

def evaluate(val_dataset, model, args, tokenizer):
    val_sampler = SequentialSampler(val_dataset)
    val_dataloader = DataLoader(val_dataset, sampler=val_sampler, batch_size=args.val_batch_size, num_workers=0)

    model.eval()
    total_loss = 0
    for batch in val_dataloader:
        input_ids = batch['input_ids'].to(device)
        descs = batch['desc'].to(device)

        with torch.no_grad():
            outputs = model(input_ids=input_ids, labels=descs)
            loss = outputs.loss
            total_loss += loss.item()

    print(f'Validation Loss: {total_loss / len(val_dataloader)}')

def generate_text(model, tokenizer, code, max_len=1024):
    model.eval()
    input_enc = tokenizer(code, max_length=max_len, padding='max_length', truncation=True, return_tensors="pt")
    input_ids = input_enc['input_ids'].to(device)

    with torch.no_grad():
        outputs = model.generate(input_ids=input_ids, max_length=max_len)

    return tokenizer.decode(outputs[0], skip_special_tokens=True)

train_dataset = CodeToTextDataset(tokenizer, args, 'train')
train(train_dataset, model, args, tokenizer)

test_dataset = CodeToTextDataset(tokenizer, args, 'test')
test(test_dataset, model, tokenizer, args)

val_dataset = CodeToTextDataset(tokenizer, args, 'eval')
evaluate(val_dataset, model, args, tokenizer)

# test_dataset = CodeToTextDataset(tokenizer, args, 'test')
# test_model(test_dataset, tokenizer, args)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/902 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/892M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.48k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/703k [00:00<?, ?B/s]

...

True Description: The vulnerability in the DynamicMetadataProvider plugin was caused by the failure to properly configure itself with MetadataFilter plugins, leading to the omission of critical security checks such as signature verification and enforcement of validity periods. This lack of essential security checks specific to deployments left the system vulnerable to potential attacks exploiting metadata vulnerabilities. The absence of these security measures within the code allowed for the possibility of unauthorized access or tampering with metadata, posing a significant risk to the overall security of the system.The vulnerability in the DynamicMetadataProvider plugin was addressed by incorporating the MetadataProvider class in the constructor's initialization list, ensuring proper configuration with MetadataFilter plugins and enabling critical security checks like signature verification and validity period enforcement. By including the MetadataProvider class in the initialization p

Testing: 100%|██████████| 416/416 [31:07<00:00,  4.49s/it]

True Description: The vulnerability in the code was caused by the improper verification of the ASN.1 metadata in the PKCS#1 v1.5 signature verification process within the `sig_verify` function. This lack of proper validation allowed a remote attacker to forge signatures, particularly when small public exponents were used, leading to the potential creation of fake X.509 certificates and enabling impersonation attacks. The absence of thorough verification of the ASN.1 metadata within the signature processing flow created a security loophole that could be exploited by malicious actors to manipulate signatures and compromise the integrity of the system.The vulnerability was addressed by introducing additional checks and validations in the `sig_verify` function. The code now includes verification steps for padding bytes, ASN.1 metadata, and adjusts expectations based on the signature type, effectively mitigating the risk of signature manipulation and impersonation attacks. By ensuring prope


Loading data: 100%|██████████| 391/391 [00:01<00:00, 282.12it/s]


Validation Loss: 1.0081417253431009


evaluation

In [9]:
import pandas as pd
import numpy as np
from transformers import RobertaTokenizer, T5ForConditionalGeneration, T5Config, AutoModelForSeq2SeqLM, AutoTokenizer, get_linear_schedule_with_warmup
from torch.utils.data import DataLoader, Dataset, RandomSampler, SequentialSampler
import torch
import torch.optim as optim
from tqdm import tqdm
import logging
import os
import spacy
from nltk.translate.bleu_score import corpus_bleu, SmoothingFunction, sentence_bleu
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import TfidfVectorizer
import textstat


logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

class Args:
    def __init__(self):
        self.train_data_file = '/content/drive/MyDrive/9900training/EqualSplitDataset/train.csv'
        self.eval_data_file = '/content/drive/MyDrive/9900training/EqualSplitDataset/val.csv'
        self.test_data_file = '/content/drive/MyDrive/9900training/EqualSplitDataset/test.csv'
        self.block_size = 512
        self.train_batch_size = 4
        self.val_batch_size = 4
        self.epochs = 10
        # # # # quick test
        # self.epochs = 1
        # self.sample_size = 50

args = Args()

model = AutoModelForSeq2SeqLM.from_pretrained('Salesforce/codet5-base-multi-sum')
tokenizer = AutoTokenizer.from_pretrained('Salesforce/codet5-base-multi-sum')

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)


class InputFeatures(object):
    def __init__(self, input_tokens, input_ids, descs):
        self.input_tokens = input_tokens
        self.input_ids = input_ids
        self.descs = descs

def convert_examples_to_features(func, desc, tokenizer, args):
    source_tokens = tokenizer.tokenize(str(func))[:args.block_size-2]
    # deleted that part because we dont know the details of t5 tokenizer, that
    # is particularlly for codebert
    source_ids = tokenizer.convert_tokens_to_ids(source_tokens)
    padding_length = args.block_size - len(source_ids)
    source_ids += [tokenizer.pad_token_id] * padding_length
    return InputFeatures(source_tokens, source_ids, str(desc))

class CodeToTextDataset(Dataset):
    def __init__(self, tokenizer, args, file_type):
        if file_type == "train":
            file_path = args.train_data_file
        elif file_type == "eval":
            file_path = args.eval_data_file
        elif file_type == "test":
            file_path = args.test_data_file

        self.examples = []
        df = pd.read_csv(file_path)
        # quick testing
        # df = pd.read_csv(file_path).sample(n=args.sample_size)
        funcs = df["vulnerable_code"].tolist()

        descs_explain = df["Explanation_of_Vulnerability_In_Context"].tolist()
        descs_change = df["Explanation_Vulnerability_Fixed_In_Context"].tolist()

        descs = []
        for i in range(len(descs_explain)):
            descs.append(str(descs_explain[i]) + str(descs_change[i]))

        for i in tqdm(range(len(funcs)), desc="Loading data"):
            self.examples.append(convert_examples_to_features(funcs[i],descs[i], tokenizer, args))
        if file_type == "train":
            for example in self.examples[:3]:
                logger.info("*** Example ***")
                logger.info("desc: {}".format(example.descs))
                logger.info("input_tokens: {}".format([x.replace('\u0120','_') for x in example.input_tokens]))
                logger.info("input_ids: {}".format(' '.join(map(str, example.input_ids))))

    def __len__(self):
        return len(self.examples)

    def __getitem__(self, idx):
        example = self.examples[idx]
        return {
            'input_ids': torch.tensor(example.input_ids, dtype=torch.long),
            'desc': torch.tensor(tokenizer.encode(example.descs, padding='max_length', truncation=True, max_length=args.block_size), dtype=torch.long),
        }


def evaluate_model(val_dataset, model, args, tokenizer):
    path_to_model = '/content/drive/MyDrive/9900training/models/description_repair_train_t5.bin'
    model.load_state_dict(torch.load(path_to_model))
    model.to(device)

    val_sampler = SequentialSampler(val_dataset)
    val_dataloader = DataLoader(val_dataset, sampler=val_sampler, batch_size=1, num_workers=0)

    model.eval()
    total_loss = 0
    bleu_scores = []
    cosine_similarities = []
    flesch_kincaid_scores = []

    tfidf_vectorizer = TfidfVectorizer()

    for batch in tqdm(val_dataloader, desc="Evaluating"):
        input_ids = batch['input_ids'].to(device)
        descs = batch['desc'].to(device)

        with torch.no_grad():
            outputs = model(input_ids=input_ids, labels=descs)
            loss = outputs.loss
            total_loss += loss.item()

            predicted_desc = tokenizer.decode(outputs.logits.argmax(-1)[0], skip_special_tokens=True)
            true_desc = tokenizer.decode(descs[0], skip_special_tokens=True)

            # BLEU-4 score
            smoothie = SmoothingFunction().method4
            bleu_score = sentence_bleu([true_desc.split()], predicted_desc.split(), smoothing_function=smoothie, weights=(0.25, 0.25, 0.25, 0.25))
            bleu_scores.append(bleu_score)

            # Cosine similarity
            tfidf_matrix = tfidf_vectorizer.fit_transform([true_desc, predicted_desc])
            cosine_sim = cosine_similarity(tfidf_matrix[0:1], tfidf_matrix[1:2])
            cosine_similarities.append(cosine_sim[0][0])

            # Flesch-Kincaid Grade Level
            flesch_kincaid_true = textstat.flesch_kincaid_grade(true_desc)
            flesch_kincaid_pred = textstat.flesch_kincaid_grade(predicted_desc)
            flesch_kincaid_diff = abs(flesch_kincaid_true - flesch_kincaid_pred)
            flesch_kincaid_scores.append(flesch_kincaid_diff)

    avg_bleu = np.mean(bleu_scores)
    avg_cosine_sim = np.mean(cosine_similarities)
    avg_flesch_kincaid = np.mean(flesch_kincaid_scores)

    print(f'Validation Loss: {total_loss / len(val_dataloader)}')
    print(f'Average BLEU-4 Score: {avg_bleu}')
    print(f'Average Cosine Similarity: {avg_cosine_sim}')
    print(f'Average Flesch-Kincaid Grade Level Difference: {avg_flesch_kincaid}')


def generate_text(model, tokenizer, code, max_len=1024):
    model.eval()
    input_enc = tokenizer(code, max_length=max_len, padding='max_length', truncation=True, return_tensors="pt")
    input_ids = input_enc['input_ids'].to(device)

    with torch.no_grad():
        outputs = model.generate(input_ids=input_ids, max_length=max_len)

    return tokenizer.decode(outputs[0], skip_special_tokens=True)

# train_dataset = CodeToTextDataset(tokenizer, args, 'train')
# train(train_dataset, model, args, tokenizer)

# test_dataset = CodeToTextDataset(tokenizer, args, 'test')
# test(test_dataset, model, tokenizer, args)

# val_dataset = CodeToTextDataset(tokenizer, args, 'eval')
# evaluate(val_dataset, model, args, tokenizer)

# test_dataset = CodeToTextDataset(tokenizer, args, 'test')
# test_model(test_dataset, tokenizer, args)

val_dataset = CodeToTextDataset(tokenizer, args, 'eval')
evaluate_model(val_dataset, model, args, tokenizer)

Evaluating: 100%|██████████| 391/391 [00:19<00:00, 19.63it/s]

Validation Loss: 1.00790679777789
Average BLEU-4 Score: 0.16657357261250122
Average Cosine Similarity: 0.7699941449615956
Average Flesch-Kincaid Grade Level Difference: 2.8104859335038364


With cwe id

In [3]:
import pandas as pd
import numpy as np
from transformers import RobertaTokenizer, T5ForConditionalGeneration, T5Config, AutoModelForSeq2SeqLM, AutoTokenizer, get_linear_schedule_with_warmup
from torch.utils.data import DataLoader, Dataset, RandomSampler, SequentialSampler
import torch
import torch.optim as optim
from tqdm import tqdm
import logging
import os
import spacy


logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

class Args:
    def __init__(self):
        self.train_data_file = '/content/drive/MyDrive/9900training/train.csv'
        self.eval_data_file = '/content/drive/MyDrive/9900training/val.csv'
        self.test_data_file = '/content/drive/MyDrive/9900training/test.csv'
        self.block_size = 512
        self.train_batch_size = 8
        self.val_batch_size = 8
        self.epochs = 10
        # # # quick test
        # self.epochs = 1
        # self.sample_size = 50

args = Args()

model = AutoModelForSeq2SeqLM.from_pretrained('Salesforce/codet5-base-multi-sum')
tokenizer = AutoTokenizer.from_pretrained('Salesforce/codet5-base-multi-sum')

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)


class InputFeatures(object):
    def __init__(self, input_tokens, input_ids, descs):
        self.input_tokens = input_tokens
        self.input_ids = input_ids
        self.descs = descs

def convert_examples_to_features(func, desc, tokenizer, args):
    source_tokens = tokenizer.tokenize(str(func))[:args.block_size-2]
    # deleted that part because we dont know the details of t5 tokenizer, that
    # is particularlly for codebert
    source_ids = tokenizer.convert_tokens_to_ids(source_tokens)
    padding_length = args.block_size - len(source_ids)
    source_ids += [tokenizer.pad_token_id] * padding_length
    return InputFeatures(source_tokens, source_ids, str(desc))

class CodeToTextDataset(Dataset):
    def __init__(self, tokenizer, args, file_type):
        if file_type == "train":
            file_path = args.train_data_file
        elif file_type == "eval":
            file_path = args.eval_data_file
        elif file_type == "test":
            file_path = args.test_data_file

        self.examples = []
        df = pd.read_csv(file_path)
        # quick testing
        # df = pd.read_csv(file_path).sample(n=args.sample_size)
        funcs = df["vulnerable_code"].tolist()
        cwe_ids = df["CWE_ID"].tolist()
        extended_funcs = []
        for i in range(len(funcs)):
            extended_funcs.append(f"code: {funcs[i]} cwe_id:{cwe_ids[i]}")

        descs = df["Explanation_of_Vulnerability_In_Context"].tolist()
        for i in tqdm(range(len(funcs)), desc="Loading data"):
            self.examples.append(convert_examples_to_features(extended_funcs[i],descs[i], tokenizer, args))
        if file_type == "train":
            for example in self.examples[:3]:
                logger.info("*** Example ***")
                logger.info("desc: {}".format(example.descs))
                logger.info("input_tokens: {}".format([x.replace('\u0120','_') for x in example.input_tokens]))
                logger.info("input_ids: {}".format(' '.join(map(str, example.input_ids))))

    def __len__(self):
        return len(self.examples)

    def __getitem__(self, idx):
        example = self.examples[idx]
        return {
            'input_ids': torch.tensor(example.input_ids, dtype=torch.long),
            'desc': torch.tensor(tokenizer.encode(example.descs, padding='max_length', truncation=True, max_length=args.block_size), dtype=torch.long),
        }

def train(train_dataset, model, args, tokenizer):
    train_sampler = RandomSampler(train_dataset)
    train_dataloader = DataLoader(train_dataset, sampler=train_sampler, batch_size=args.train_batch_size, num_workers=0)

    optimizer = optim.AdamW(model.parameters(), lr=5e-5)
    total_steps = len(train_dataloader) * args.epochs
    scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=0, num_training_steps=total_steps)

    for epoch in range(args.epochs):
        model.train()
        for batch in train_dataloader:
            input_ids = batch['input_ids'].to(device)
            descs = batch['desc'].to(device)

            outputs = model(input_ids=input_ids, labels=descs)
            loss = outputs.loss

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            scheduler.step()

            print(f'Epoch: {epoch}, Loss: {loss.item()}')
    model_to_save = model.module if hasattr(model,'module') else model
    torch.save(model_to_save.state_dict(), '/content/drive/MyDrive/9900training/custom_cweid_train_t5.bin')
    # torch.save(model_to_save.state_dict(), '/content/drive/MyDrive/9900training/custom_train_t5.bin')


def test(test_dataset, model, tokenizer, args):
    test_sampler = SequentialSampler(test_dataset)
    test_dataloader = DataLoader(test_dataset, sampler=test_sampler, batch_size=1, num_workers=0)

    model.eval()
    total_loss = 0
    for batch in tqdm(test_dataloader, desc="Testing"):
        input_ids = batch['input_ids'].to(device)
        true_desc = batch['desc'].to(device)

        with torch.no_grad():
            outputs = model.generate(input_ids=input_ids, max_length=args.block_size)

        predicted_desc = tokenizer.decode(outputs[0], skip_special_tokens=True)
        true_desc_decoded = tokenizer.decode(true_desc[0], skip_special_tokens=True)

        logger.info(f"True Description: {true_desc_decoded}")
        logger.info(f"Predicted Description: {predicted_desc}")
        print(f"True Description: {true_desc_decoded}")
        print(f"Predicted Description: {predicted_desc}")


def test_model(test_dataset, tokenizer, args):
    path_to_model = '/content/drive/MyDrive/9900training/custom_cweid_train_t5.bin'
    model.load_state_dict(torch.load(path_to_model))
    test_sampler = SequentialSampler(test_dataset)
    test_dataloader = DataLoader(test_dataset, sampler=test_sampler, batch_size=1, num_workers=0)

    model.eval()
    total_loss = 0
    for batch in tqdm(test_dataloader, desc="Testing"):
        input_ids = batch['input_ids'].to(device)
        true_desc = batch['desc'].to(device)

        with torch.no_grad():
            outputs = model.generate(input_ids=input_ids, max_length=args.block_size)

        predicted_desc = tokenizer.decode(outputs[0], skip_special_tokens=True)
        true_desc_decoded = tokenizer.decode(true_desc[0], skip_special_tokens=True)

        logger.info(f"True Description: {true_desc_decoded}")
        logger.info(f"Predicted Description: {predicted_desc}")
        print(f"True Description: {true_desc_decoded}")
        print(f"Predicted Description: {predicted_desc}")

def evaluate(val_dataset, model, args, tokenizer):
    val_sampler = SequentialSampler(val_dataset)
    val_dataloader = DataLoader(val_dataset, sampler=val_sampler, batch_size=args.val_batch_size, num_workers=0)

    model.eval()
    total_loss = 0
    for batch in val_dataloader:
        input_ids = batch['input_ids'].to(device)
        descs = batch['desc'].to(device)

        with torch.no_grad():
            outputs = model(input_ids=input_ids, labels=descs)
            loss = outputs.loss
            total_loss += loss.item()

    print(f'Validation Loss: {total_loss / len(val_dataloader)}')

def generate_text(model, tokenizer, code, max_len=1024):
    model.eval()
    input_enc = tokenizer(code, max_length=max_len, padding='max_length', truncation=True, return_tensors="pt")
    input_ids = input_enc['input_ids'].to(device)

    with torch.no_grad():
        outputs = model.generate(input_ids=input_ids, max_length=max_len)

    return tokenizer.decode(outputs[0], skip_special_tokens=True)

train_dataset = CodeToTextDataset(tokenizer, args, 'train')
train(train_dataset, model, args, tokenizer)

test_dataset = CodeToTextDataset(tokenizer, args, 'test')
test(test_dataset, model, tokenizer, args)

val_dataset = CodeToTextDataset(tokenizer, args, 'eval')
evaluate(val_dataset, model, args, tokenizer)

# test_dataset = CodeToTextDataset(tokenizer, args, 'test')
# test_model(test_dataset, tokenizer, args)

Loading data: 100%|██████████| 448/448 [00:01<00:00, 290.87it/s]


Epoch: 0, Loss: 2.8509819507598877
Epoch: 0, Loss: 1.3939529657363892
Epoch: 0, Loss: 1.372054100036621
Epoch: 0, Loss: 1.266452670097351
Epoch: 0, Loss: 1.1933037042617798
Epoch: 0, Loss: 1.2673975229263306
Epoch: 0, Loss: 1.1746512651443481
Epoch: 0, Loss: 1.1608680486679077
Epoch: 0, Loss: 1.260926604270935
Epoch: 0, Loss: 1.1059520244598389
Epoch: 0, Loss: 1.0407469272613525
Epoch: 0, Loss: 1.1374911069869995
Epoch: 0, Loss: 1.0495636463165283
Epoch: 0, Loss: 1.142653465270996
Epoch: 0, Loss: 1.1038689613342285
Epoch: 0, Loss: 1.016004204750061
Epoch: 0, Loss: 0.9731031060218811
Epoch: 0, Loss: 0.9498294591903687
Epoch: 0, Loss: 0.8781737089157104
Epoch: 0, Loss: 0.9271970987319946
Epoch: 0, Loss: 1.0391556024551392
Epoch: 0, Loss: 0.8687582015991211
Epoch: 0, Loss: 0.9423884749412537
Epoch: 0, Loss: 0.9082649350166321
Epoch: 0, Loss: 0.8978636264801025
Epoch: 0, Loss: 0.7553812265396118
Epoch: 0, Loss: 0.7142033576965332
Epoch: 0, Loss: 0.9095907211303711
Epoch: 0, Loss: 0.9507197

Testing:   0%|          | 1/416 [00:02<14:44,  2.13s/it]

True Description: The vulnerability was caused by the absence of an End of File (EOF) check within the loop that reads data from the image blob. Specifically, in the `ReadPSImage` function, the loop iterates based on the `length` variable, which is derived from the `extent` field in the header. If a crafted file claims a large `extent` but does not provide sufficient data, the loop continues indefinitely, leading to excessive CPU consumption as it attempts to read beyond the available data. This oversight allows an attacker to create a denial-of-service condition by exploiting the lack of proper bounds checking.
Predicted Description: The vulnerability in the `ReadPSImage` function was caused by a lack of proper validation for the `next` variable in the `read_info` variable before performing operations that require a specific value for `next` before it was called. This oversight allowed local users to exploit the vulnerability by providing a crafted XHR file that triggered the `next` v

Testing:   0%|          | 2/416 [00:04<14:59,  2.17s/it]

...

True Description: The vulnerability in the DynamicMetadataProvider plugin was caused by the failure to properly configure itself with MetadataFilter plugins, leading to the omission of critical security checks such as signature verification and enforcement of validity periods. This lack of essential security checks specific to deployments left the system vulnerable to potential attacks exploiting metadata vulnerabilities. The absence of these security measures within the code allowed for the possibility of unauthorized access or tampering with metadata, posing a significant risk to the overall security of the system.
Predicted Description: The vulnerability in the DynamicMetadataProvider class was caused by a lack of proper validation for the `m_match` and `m_subst` attributes, allowing remote attackers to manipulate the metadata in a way that triggers the lack of proper validation and validation of the `m_match` attribute. This oversight allowed remote attackers to exploit the vulnera

Testing: 100%|██████████| 416/416 [15:24<00:00,  2.22s/it]


True Description: The vulnerability in the code was caused by the improper verification of the ASN.1 metadata in the PKCS#1 v1.5 signature verification process within the `sig_verify` function. This lack of proper validation allowed a remote attacker to forge signatures, particularly when small public exponents were used, leading to the potential creation of fake X.509 certificates and enabling impersonation attacks. The absence of thorough verification of the ASN.1 metadata within the signature processing flow created a security loophole that could be exploited by malicious actors to manipulate signatures and compromise the integrity of the system.
Predicted Description: The vulnerability in the `sig_verify` function was caused by the lack of proper memory management when processing a signature. This oversight allowed remote attackers to exploit the vulnerability by providing a crafted RSA key in the RSA key signature. The lack of proper memory management allowed remote attackers to e

Loading data: 100%|██████████| 391/391 [00:01<00:00, 249.79it/s]


Validation Loss: 0.5673610847823474


evaluation

In [7]:
import pandas as pd
import numpy as np
from transformers import RobertaTokenizer, T5ForConditionalGeneration, T5Config, AutoModelForSeq2SeqLM, AutoTokenizer, get_linear_schedule_with_warmup
from torch.utils.data import DataLoader, Dataset, RandomSampler, SequentialSampler
import torch
import torch.optim as optim
from tqdm import tqdm
import logging
import os
import spacy
from nltk.translate.bleu_score import corpus_bleu, SmoothingFunction, sentence_bleu
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import TfidfVectorizer
import textstat


logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

class Args:
    def __init__(self):
        self.train_data_file = '/content/drive/MyDrive/9900training/EqualSplitDataset/train.csv'
        self.eval_data_file = '/content/drive/MyDrive/9900training/EqualSplitDataset/val.csv'
        self.test_data_file = '/content/drive/MyDrive/9900training/EqualSplitDataset/test.csv'
        self.block_size = 512
        self.train_batch_size = 4
        self.val_batch_size = 4
        self.epochs = 10
        # # # # quick test
        # self.epochs = 1
        # self.sample_size = 50

args = Args()

model = AutoModelForSeq2SeqLM.from_pretrained('Salesforce/codet5-base-multi-sum')
tokenizer = AutoTokenizer.from_pretrained('Salesforce/codet5-base-multi-sum')

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)


class InputFeatures(object):
    def __init__(self, input_tokens, input_ids, descs):
        self.input_tokens = input_tokens
        self.input_ids = input_ids
        self.descs = descs

def convert_examples_to_features(func, desc, tokenizer, args):
    source_tokens = tokenizer.tokenize(str(func))[:args.block_size-2]
    # deleted that part because we dont know the details of t5 tokenizer, that
    # is particularlly for codebert
    source_ids = tokenizer.convert_tokens_to_ids(source_tokens)
    padding_length = args.block_size - len(source_ids)
    source_ids += [tokenizer.pad_token_id] * padding_length
    return InputFeatures(source_tokens, source_ids, str(desc))

class CodeToTextDataset(Dataset):
    def __init__(self, tokenizer, args, file_type):
        if file_type == "train":
            file_path = args.train_data_file
        elif file_type == "eval":
            file_path = args.eval_data_file
        elif file_type == "test":
            file_path = args.test_data_file

        self.examples = []
        df = pd.read_csv(file_path)
        # quick testing
        # df = pd.read_csv(file_path).sample(n=args.sample_size)
        funcs = df["vulnerable_code"].tolist()

        descs_explain = df["Explanation_of_Vulnerability_In_Context"].tolist()
        descs_change = df["Explanation_Vulnerability_Fixed_In_Context"].tolist()

        descs = []
        for i in range(len(descs_explain)):
            descs.append(str(descs_explain[i]) + str(descs_change[i]))

        for i in tqdm(range(len(funcs)), desc="Loading data"):
            self.examples.append(convert_examples_to_features(funcs[i],descs[i], tokenizer, args))
        if file_type == "train":
            for example in self.examples[:3]:
                logger.info("*** Example ***")
                logger.info("desc: {}".format(example.descs))
                logger.info("input_tokens: {}".format([x.replace('\u0120','_') for x in example.input_tokens]))
                logger.info("input_ids: {}".format(' '.join(map(str, example.input_ids))))

    def __len__(self):
        return len(self.examples)

    def __getitem__(self, idx):
        example = self.examples[idx]
        return {
            'input_ids': torch.tensor(example.input_ids, dtype=torch.long),
            'desc': torch.tensor(tokenizer.encode(example.descs, padding='max_length', truncation=True, max_length=args.block_size), dtype=torch.long),
        }


def evaluate_model(val_dataset, model, args, tokenizer):
    path_to_model = '/content/drive/MyDrive/9900training/models/custom_cweid_train_t5.bin'
    model.load_state_dict(torch.load(path_to_model))
    model.to(device)

    val_sampler = SequentialSampler(val_dataset)
    val_dataloader = DataLoader(val_dataset, sampler=val_sampler, batch_size=1, num_workers=0)

    model.eval()
    total_loss = 0
    bleu_scores = []
    cosine_similarities = []
    flesch_kincaid_scores = []

    tfidf_vectorizer = TfidfVectorizer()

    for batch in tqdm(val_dataloader, desc="Evaluating"):
        input_ids = batch['input_ids'].to(device)
        descs = batch['desc'].to(device)

        with torch.no_grad():
            outputs = model(input_ids=input_ids, labels=descs)
            loss = outputs.loss
            total_loss += loss.item()

            predicted_desc = tokenizer.decode(outputs.logits.argmax(-1)[0], skip_special_tokens=True)
            true_desc = tokenizer.decode(descs[0], skip_special_tokens=True)

            # BLEU-4 score
            smoothie = SmoothingFunction().method4
            bleu_score = sentence_bleu([true_desc.split()], predicted_desc.split(), smoothing_function=smoothie, weights=(0.25, 0.25, 0.25, 0.25))
            bleu_scores.append(bleu_score)

            # Cosine similarity
            tfidf_matrix = tfidf_vectorizer.fit_transform([true_desc, predicted_desc])
            cosine_sim = cosine_similarity(tfidf_matrix[0:1], tfidf_matrix[1:2])
            cosine_similarities.append(cosine_sim[0][0])

            # Flesch-Kincaid Grade Level
            flesch_kincaid_true = textstat.flesch_kincaid_grade(true_desc)
            flesch_kincaid_pred = textstat.flesch_kincaid_grade(predicted_desc)
            flesch_kincaid_diff = abs(flesch_kincaid_true - flesch_kincaid_pred)
            flesch_kincaid_scores.append(flesch_kincaid_diff)

    avg_bleu = np.mean(bleu_scores)
    avg_cosine_sim = np.mean(cosine_similarities)
    avg_flesch_kincaid = np.mean(flesch_kincaid_scores)

    print(f'Validation Loss: {total_loss / len(val_dataloader)}')
    print(f'Average BLEU-4 Score: {avg_bleu}')
    print(f'Average Cosine Similarity: {avg_cosine_sim}')
    print(f'Average Flesch-Kincaid Grade Level Difference: {avg_flesch_kincaid}')


def generate_text(model, tokenizer, code, max_len=1024):
    model.eval()
    input_enc = tokenizer(code, max_length=max_len, padding='max_length', truncation=True, return_tensors="pt")
    input_ids = input_enc['input_ids'].to(device)

    with torch.no_grad():
        outputs = model.generate(input_ids=input_ids, max_length=max_len)

    return tokenizer.decode(outputs[0], skip_special_tokens=True)

# train_dataset = CodeToTextDataset(tokenizer, args, 'train')
# train(train_dataset, model, args, tokenizer)

# test_dataset = CodeToTextDataset(tokenizer, args, 'test')
# test(test_dataset, model, tokenizer, args)

# val_dataset = CodeToTextDataset(tokenizer, args, 'eval')
# evaluate(val_dataset, model, args, tokenizer)

# test_dataset = CodeToTextDataset(tokenizer, args, 'test')
# test_model(test_dataset, tokenizer, args)

val_dataset = CodeToTextDataset(tokenizer, args, 'eval')
evaluate_model(val_dataset, model, args, tokenizer)

Evaluating: 100%|██████████| 391/391 [00:15<00:00, 25.43it/s]

Validation Loss: 1.336222140704427
Average BLEU-4 Score: 0.09847627396747653
Average Cosine Similarity: 0.7217340780267022
Average Flesch-Kincaid Grade Level Difference: 3.8588235294117648


Text generation v1

In [ ]:
import pandas as pd
import numpy as np
from transformers import RobertaTokenizer, T5ForConditionalGeneration, T5Config, AutoModelForSeq2SeqLM, AutoTokenizer, get_linear_schedule_with_warmup
from torch.utils.data import DataLoader, Dataset, RandomSampler, SequentialSampler
import torch
import torch.optim as optim
from tqdm import tqdm
import logging
import os
import spacy


logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

class Args:
    def __init__(self):
        self.train_data_file = '/content/drive/MyDrive/9900training/train.csv'
        self.eval_data_file = '/content/drive/MyDrive/9900training/val.csv'
        self.test_data_file = '/content/drive/MyDrive/9900training/test.csv'
        self.block_size = 512
        self.train_batch_size = 8
        self.val_batch_size = 8
        # self.epochs = 10
        # # quick test
        self.epochs = 1
        self.sample_size = 50

args = Args()

model = AutoModelForSeq2SeqLM.from_pretrained('Salesforce/codet5-base-multi-sum')
tokenizer = AutoTokenizer.from_pretrained('Salesforce/codet5-base-multi-sum')

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)


class InputFeatures(object):
    def __init__(self, input_tokens, input_ids, descs):
        self.input_tokens = input_tokens
        self.input_ids = input_ids
        self.descs = descs

def convert_examples_to_features(func, desc, tokenizer, args):
    source_tokens = tokenizer.tokenize(str(func))[:args.block_size-2]
    # deleted that part because we dont know the details of t5 tokenizer, that
    # is particularlly for codebert
    source_ids = tokenizer.convert_tokens_to_ids(source_tokens)
    padding_length = args.block_size - len(source_ids)
    source_ids += [tokenizer.pad_token_id] * padding_length
    return InputFeatures(source_tokens, source_ids, str(desc))

class CodeToTextDataset(Dataset):
    def __init__(self, tokenizer, args, file_type):
        if file_type == "train":
            file_path = args.train_data_file
        elif file_type == "eval":
            file_path = args.eval_data_file
        elif file_type == "test":
            file_path = args.test_data_file

        self.examples = []
        # df = pd.read_csv(file_path)
        # quick testing
        df = pd.read_csv(file_path).sample(n=args.sample_size)
        funcs = df["vulnerable_code"].tolist()
        descs = df["Explanation_of_Vulnerability_In_Context"].tolist()
        for i in tqdm(range(len(funcs)), desc="Loading data"):
            self.examples.append(convert_examples_to_features(funcs[i],descs[i], tokenizer, args))
        if file_type == "train":
            for example in self.examples[:3]:
                logger.info("*** Example ***")
                logger.info("desc: {}".format(example.descs))
                logger.info("input_tokens: {}".format([x.replace('\u0120','_') for x in example.input_tokens]))
                logger.info("input_ids: {}".format(' '.join(map(str, example.input_ids))))

    def __len__(self):
        return len(self.examples)

    def __getitem__(self, idx):
        example = self.examples[idx]
        return {
            'input_ids': torch.tensor(example.input_ids, dtype=torch.long),
            'desc': torch.tensor(tokenizer.encode(example.descs, padding='max_length', truncation=True, max_length=args.block_size), dtype=torch.long),
        }

def train(train_dataset, model, args, tokenizer):
    train_sampler = RandomSampler(train_dataset)
    train_dataloader = DataLoader(train_dataset, sampler=train_sampler, batch_size=args.train_batch_size, num_workers=0)

    optimizer = optim.AdamW(model.parameters(), lr=5e-5)
    total_steps = len(train_dataloader) * args.epochs
    scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=0, num_training_steps=total_steps)

    for epoch in range(args.epochs):
        model.train()
        for batch in train_dataloader:
            input_ids = batch['input_ids'].to(device)
            descs = batch['desc'].to(device)

            outputs = model(input_ids=input_ids, labels=descs)
            loss = outputs.loss

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            scheduler.step()

            print(f'Epoch: {epoch}, Loss: {loss.item()}')
    model_to_save = model.module if hasattr(model,'module') else model
    torch.save(model_to_save.state_dict(), '/content/drive/MyDrive/9900training/custom_train_t5.bin')


def test(test_dataset, model, tokenizer, args):
    test_sampler = SequentialSampler(test_dataset)
    test_dataloader = DataLoader(test_dataset, sampler=test_sampler, batch_size=1, num_workers=0)

    model.eval()
    total_loss = 0
    for batch in tqdm(test_dataloader, desc="Testing"):
        input_ids = batch['input_ids'].to(device)
        true_desc = batch['desc'].to(device)

        with torch.no_grad():
            outputs = model.generate(input_ids=input_ids, max_length=args.block_size)

        predicted_desc = tokenizer.decode(outputs[0], skip_special_tokens=True)
        true_desc_decoded = tokenizer.decode(true_desc[0], skip_special_tokens=True)

        logger.info(f"True Description: {true_desc_decoded}")
        logger.info(f"Predicted Description: {predicted_desc}")
        print(f"True Description: {true_desc_decoded}")
        print(f"Predicted Description: {predicted_desc}")


def test_model(test_dataset, tokenizer, args):
    path_to_model = '/content/drive/MyDrive/9900training/custom_train_t5.bin'
    model.load_state_dict(torch.load(path_to_model))
    test_sampler = SequentialSampler(test_dataset)
    test_dataloader = DataLoader(test_dataset, sampler=test_sampler, batch_size=1, num_workers=0)

    model.eval()
    total_loss = 0
    for batch in tqdm(test_dataloader, desc="Testing"):
        input_ids = batch['input_ids'].to(device)
        true_desc = batch['desc'].to(device)

        with torch.no_grad():
            outputs = model.generate(input_ids=input_ids, max_length=args.block_size)

        predicted_desc = tokenizer.decode(outputs[0], skip_special_tokens=True)
        true_desc_decoded = tokenizer.decode(true_desc[0], skip_special_tokens=True)

        logger.info(f"True Description: {true_desc_decoded}")
        logger.info(f"Predicted Description: {predicted_desc}")
        print(f"True Description: {true_desc_decoded}")
        print(f"Predicted Description: {predicted_desc}")

def evaluate(val_dataset, model, args, tokenizer):
    val_sampler = SequentialSampler(val_dataset)
    val_dataloader = DataLoader(val_dataset, sampler=val_sampler, batch_size=args.val_batch_size, num_workers=0)

    model.eval()
    total_loss = 0
    for batch in val_dataloader:
        input_ids = batch['input_ids'].to(device)
        descs = batch['desc'].to(device)

        with torch.no_grad():
            outputs = model(input_ids=input_ids, labels=descs)
            loss = outputs.loss
            total_loss += loss.item()

    print(f'Validation Loss: {total_loss / len(val_dataloader)}')

def generate_text(model, tokenizer, code, max_len=1024):
    model.eval()
    input_enc = tokenizer(code, max_length=max_len, padding='max_length', truncation=True, return_tensors="pt")
    input_ids = input_enc['input_ids'].to(device)

    with torch.no_grad():
        outputs = model.generate(input_ids=input_ids, max_length=max_len)

    return tokenizer.decode(outputs[0], skip_special_tokens=True)

train_dataset = CodeToTextDataset(tokenizer, args, 'train')
train(train_dataset, model, args, tokenizer)

test_dataset = CodeToTextDataset(tokenizer, args, 'test')
test(test_dataset, model, tokenizer, args)

val_dataset = CodeToTextDataset(tokenizer, args, 'eval')
evaluate(val_dataset, model, args, tokenizer)

# test_dataset = CodeToTextDataset(tokenizer, args, 'test')
# test_model(test_dataset, tokenizer, args)

evaluation

In [8]:
import pandas as pd
import numpy as np
from transformers import RobertaTokenizer, T5ForConditionalGeneration, T5Config, AutoModelForSeq2SeqLM, AutoTokenizer, get_linear_schedule_with_warmup
from torch.utils.data import DataLoader, Dataset, RandomSampler, SequentialSampler
import torch
import torch.optim as optim
from tqdm import tqdm
import logging
import os
import spacy
from nltk.translate.bleu_score import corpus_bleu, SmoothingFunction, sentence_bleu
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import TfidfVectorizer
import textstat


logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

class Args:
    def __init__(self):
        self.train_data_file = '/content/drive/MyDrive/9900training/EqualSplitDataset/train.csv'
        self.eval_data_file = '/content/drive/MyDrive/9900training/EqualSplitDataset/val.csv'
        self.test_data_file = '/content/drive/MyDrive/9900training/EqualSplitDataset/test.csv'
        self.block_size = 512
        self.train_batch_size = 4
        self.val_batch_size = 4
        self.epochs = 10
        # # # # quick test
        # self.epochs = 1
        # self.sample_size = 50

args = Args()

model = AutoModelForSeq2SeqLM.from_pretrained('Salesforce/codet5-base-multi-sum')
tokenizer = AutoTokenizer.from_pretrained('Salesforce/codet5-base-multi-sum')

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)


class InputFeatures(object):
    def __init__(self, input_tokens, input_ids, descs):
        self.input_tokens = input_tokens
        self.input_ids = input_ids
        self.descs = descs

def convert_examples_to_features(func, desc, tokenizer, args):
    source_tokens = tokenizer.tokenize(str(func))[:args.block_size-2]
    # deleted that part because we dont know the details of t5 tokenizer, that
    # is particularlly for codebert
    source_ids = tokenizer.convert_tokens_to_ids(source_tokens)
    padding_length = args.block_size - len(source_ids)
    source_ids += [tokenizer.pad_token_id] * padding_length
    return InputFeatures(source_tokens, source_ids, str(desc))

class CodeToTextDataset(Dataset):
    def __init__(self, tokenizer, args, file_type):
        if file_type == "train":
            file_path = args.train_data_file
        elif file_type == "eval":
            file_path = args.eval_data_file
        elif file_type == "test":
            file_path = args.test_data_file

        self.examples = []
        df = pd.read_csv(file_path)
        # quick testing
        # df = pd.read_csv(file_path).sample(n=args.sample_size)
        funcs = df["vulnerable_code"].tolist()

        descs_explain = df["Explanation_of_Vulnerability_In_Context"].tolist()
        descs_change = df["Explanation_Vulnerability_Fixed_In_Context"].tolist()

        descs = []
        for i in range(len(descs_explain)):
            descs.append(str(descs_explain[i]) + str(descs_change[i]))

        for i in tqdm(range(len(funcs)), desc="Loading data"):
            self.examples.append(convert_examples_to_features(funcs[i],descs[i], tokenizer, args))
        if file_type == "train":
            for example in self.examples[:3]:
                logger.info("*** Example ***")
                logger.info("desc: {}".format(example.descs))
                logger.info("input_tokens: {}".format([x.replace('\u0120','_') for x in example.input_tokens]))
                logger.info("input_ids: {}".format(' '.join(map(str, example.input_ids))))

    def __len__(self):
        return len(self.examples)

    def __getitem__(self, idx):
        example = self.examples[idx]
        return {
            'input_ids': torch.tensor(example.input_ids, dtype=torch.long),
            'desc': torch.tensor(tokenizer.encode(example.descs, padding='max_length', truncation=True, max_length=args.block_size), dtype=torch.long),
        }


def evaluate_model(val_dataset, model, args, tokenizer):
    path_to_model = '/content/drive/MyDrive/9900training/models/desc_model.bin'
    model.load_state_dict(torch.load(path_to_model))
    model.to(device)

    val_sampler = SequentialSampler(val_dataset)
    val_dataloader = DataLoader(val_dataset, sampler=val_sampler, batch_size=1, num_workers=0)

    model.eval()
    total_loss = 0
    bleu_scores = []
    cosine_similarities = []
    flesch_kincaid_scores = []

    tfidf_vectorizer = TfidfVectorizer()

    for batch in tqdm(val_dataloader, desc="Evaluating"):
        input_ids = batch['input_ids'].to(device)
        descs = batch['desc'].to(device)

        with torch.no_grad():
            outputs = model(input_ids=input_ids, labels=descs)
            loss = outputs.loss
            total_loss += loss.item()

            predicted_desc = tokenizer.decode(outputs.logits.argmax(-1)[0], skip_special_tokens=True)
            true_desc = tokenizer.decode(descs[0], skip_special_tokens=True)

            # BLEU-4 score
            smoothie = SmoothingFunction().method4
            bleu_score = sentence_bleu([true_desc.split()], predicted_desc.split(), smoothing_function=smoothie, weights=(0.25, 0.25, 0.25, 0.25))
            bleu_scores.append(bleu_score)

            # Cosine similarity
            tfidf_matrix = tfidf_vectorizer.fit_transform([true_desc, predicted_desc])
            cosine_sim = cosine_similarity(tfidf_matrix[0:1], tfidf_matrix[1:2])
            cosine_similarities.append(cosine_sim[0][0])

            # Flesch-Kincaid Grade Level
            flesch_kincaid_true = textstat.flesch_kincaid_grade(true_desc)
            flesch_kincaid_pred = textstat.flesch_kincaid_grade(predicted_desc)
            flesch_kincaid_diff = abs(flesch_kincaid_true - flesch_kincaid_pred)
            flesch_kincaid_scores.append(flesch_kincaid_diff)

    avg_bleu = np.mean(bleu_scores)
    avg_cosine_sim = np.mean(cosine_similarities)
    avg_flesch_kincaid = np.mean(flesch_kincaid_scores)

    print(f'Validation Loss: {total_loss / len(val_dataloader)}')
    print(f'Average BLEU-4 Score: {avg_bleu}')
    print(f'Average Cosine Similarity: {avg_cosine_sim}')
    print(f'Average Flesch-Kincaid Grade Level Difference: {avg_flesch_kincaid}')


def generate_text(model, tokenizer, code, max_len=1024):
    model.eval()
    input_enc = tokenizer(code, max_length=max_len, padding='max_length', truncation=True, return_tensors="pt")
    input_ids = input_enc['input_ids'].to(device)

    with torch.no_grad():
        outputs = model.generate(input_ids=input_ids, max_length=max_len)

    return tokenizer.decode(outputs[0], skip_special_tokens=True)

# train_dataset = CodeToTextDataset(tokenizer, args, 'train')
# train(train_dataset, model, args, tokenizer)

# test_dataset = CodeToTextDataset(tokenizer, args, 'test')
# test(test_dataset, model, tokenizer, args)

# val_dataset = CodeToTextDataset(tokenizer, args, 'eval')
# evaluate(val_dataset, model, args, tokenizer)

# test_dataset = CodeToTextDataset(tokenizer, args, 'test')
# test_model(test_dataset, tokenizer, args)

val_dataset = CodeToTextDataset(tokenizer, args, 'eval')
evaluate_model(val_dataset, model, args, tokenizer)

Evaluating: 100%|██████████| 391/391 [00:14<00:00, 26.15it/s]

Validation Loss: 1.3359015386770754
Average BLEU-4 Score: 0.09852709232158907
Average Cosine Similarity: 0.7215431974004194
Average Flesch-Kincaid Grade Level Difference: 3.928132992327366


Evaluate untrained code-to-text T5 model

In [9]:
import pandas as pd
import numpy as np
from transformers import RobertaTokenizer, T5ForConditionalGeneration, T5Config, AutoModelForSeq2SeqLM, AutoTokenizer, get_linear_schedule_with_warmup
from torch.utils.data import DataLoader, Dataset, RandomSampler, SequentialSampler
import torch
import torch.optim as optim
from tqdm import tqdm
import logging
import os
import spacy
from nltk.translate.bleu_score import corpus_bleu, SmoothingFunction, sentence_bleu
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import TfidfVectorizer
import textstat


logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

class Args:
    def __init__(self):
        self.train_data_file = '/content/drive/MyDrive/9900training/EqualSplitDataset/train.csv'
        self.eval_data_file = '/content/drive/MyDrive/9900training/EqualSplitDataset/val.csv'
        self.test_data_file = '/content/drive/MyDrive/9900training/EqualSplitDataset/test.csv'
        self.block_size = 512
        self.train_batch_size = 4
        self.val_batch_size = 4
        self.epochs = 10
        # # # # quick test
        # self.epochs = 1
        # self.sample_size = 50

args = Args()

model = AutoModelForSeq2SeqLM.from_pretrained('Salesforce/codet5-base-multi-sum')
tokenizer = AutoTokenizer.from_pretrained('Salesforce/codet5-base-multi-sum')

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)


class InputFeatures(object):
    def __init__(self, input_tokens, input_ids, descs):
        self.input_tokens = input_tokens
        self.input_ids = input_ids
        self.descs = descs

def convert_examples_to_features(func, desc, tokenizer, args):
    source_tokens = tokenizer.tokenize(str(func))[:args.block_size-2]
    # deleted that part because we dont know the details of t5 tokenizer, that
    # is particularlly for codebert
    source_ids = tokenizer.convert_tokens_to_ids(source_tokens)
    padding_length = args.block_size - len(source_ids)
    source_ids += [tokenizer.pad_token_id] * padding_length
    return InputFeatures(source_tokens, source_ids, str(desc))

class CodeToTextDataset(Dataset):
    def __init__(self, tokenizer, args, file_type):
        if file_type == "train":
            file_path = args.train_data_file
        elif file_type == "eval":
            file_path = args.eval_data_file
        elif file_type == "test":
            file_path = args.test_data_file

        self.examples = []
        df = pd.read_csv(file_path)
        # quick testing
        # df = pd.read_csv(file_path).sample(n=args.sample_size)
        funcs = df["vulnerable_code"].tolist()

        descs_explain = df["Explanation_of_Vulnerability_In_Context"].tolist()
        descs_change = df["Explanation_Vulnerability_Fixed_In_Context"].tolist()

        descs = []
        for i in range(len(descs_explain)):
            descs.append(str(descs_explain[i]) + str(descs_change[i]))

        for i in tqdm(range(len(funcs)), desc="Loading data"):
            self.examples.append(convert_examples_to_features(funcs[i],descs[i], tokenizer, args))
        if file_type == "train":
            for example in self.examples[:3]:
                logger.info("*** Example ***")
                logger.info("desc: {}".format(example.descs))
                logger.info("input_tokens: {}".format([x.replace('\u0120','_') for x in example.input_tokens]))
                logger.info("input_ids: {}".format(' '.join(map(str, example.input_ids))))

    def __len__(self):
        return len(self.examples)

    def __getitem__(self, idx):
        example = self.examples[idx]
        return {
            'input_ids': torch.tensor(example.input_ids, dtype=torch.long),
            'desc': torch.tensor(tokenizer.encode(example.descs, padding='max_length', truncation=True, max_length=args.block_size), dtype=torch.long),
        }


def evaluate_model(val_dataset, model, args, tokenizer):
    model = AutoModelForSeq2SeqLM.from_pretrained('Salesforce/codet5-base-multi-sum')
    model.to(device)

    val_sampler = SequentialSampler(val_dataset)
    val_dataloader = DataLoader(val_dataset, sampler=val_sampler, batch_size=1, num_workers=0)

    model.eval()
    total_loss = 0
    bleu_scores = []
    cosine_similarities = []
    flesch_kincaid_scores = []

    tfidf_vectorizer = TfidfVectorizer()

    for batch in tqdm(val_dataloader, desc="Evaluating"):
        input_ids = batch['input_ids'].to(device)
        descs = batch['desc'].to(device)

        with torch.no_grad():
            outputs = model(input_ids=input_ids, labels=descs)
            loss = outputs.loss
            total_loss += loss.item()

            predicted_desc = tokenizer.decode(outputs.logits.argmax(-1)[0], skip_special_tokens=True)
            true_desc = tokenizer.decode(descs[0], skip_special_tokens=True)

            # BLEU-4 score
            smoothie = SmoothingFunction().method4
            bleu_score = sentence_bleu([true_desc.split()], predicted_desc.split(), smoothing_function=smoothie, weights=(0.25, 0.25, 0.25, 0.25))
            bleu_scores.append(bleu_score)

            # Cosine similarity
            tfidf_matrix = tfidf_vectorizer.fit_transform([true_desc, predicted_desc])
            cosine_sim = cosine_similarity(tfidf_matrix[0:1], tfidf_matrix[1:2])
            cosine_similarities.append(cosine_sim[0][0])

            # Flesch-Kincaid Grade Level
            flesch_kincaid_true = textstat.flesch_kincaid_grade(true_desc)
            flesch_kincaid_pred = textstat.flesch_kincaid_grade(predicted_desc)
            flesch_kincaid_diff = abs(flesch_kincaid_true - flesch_kincaid_pred)
            flesch_kincaid_scores.append(flesch_kincaid_diff)

    avg_bleu = np.mean(bleu_scores)
    avg_cosine_sim = np.mean(cosine_similarities)
    avg_flesch_kincaid = np.mean(flesch_kincaid_scores)

    print(f'Validation Loss: {total_loss / len(val_dataloader)}')
    print(f'Average BLEU-4 Score: {avg_bleu}')
    print(f'Average Cosine Similarity: {avg_cosine_sim}')
    print(f'Average Flesch-Kincaid Grade Level Difference: {avg_flesch_kincaid}')


def generate_text(model, tokenizer, code, max_len=1024):
    model.eval()
    input_enc = tokenizer(code, max_length=max_len, padding='max_length', truncation=True, return_tensors="pt")
    input_ids = input_enc['input_ids'].to(device)

    with torch.no_grad():
        outputs = model.generate(input_ids=input_ids, max_length=max_len)

    return tokenizer.decode(outputs[0], skip_special_tokens=True)

# train_dataset = CodeToTextDataset(tokenizer, args, 'train')
# train(train_dataset, model, args, tokenizer)

# test_dataset = CodeToTextDataset(tokenizer, args, 'test')
# test(test_dataset, model, tokenizer, args)

# val_dataset = CodeToTextDataset(tokenizer, args, 'eval')
# evaluate(val_dataset, model, args, tokenizer)

# test_dataset = CodeToTextDataset(tokenizer, args, 'test')
# test_model(test_dataset, tokenizer, args)

val_dataset = CodeToTextDataset(tokenizer, args, 'eval')
evaluate_model(val_dataset, model, args, tokenizer)

Evaluating: 100%|██████████| 391/391 [00:15<00:00, 25.45it/s]

Validation Loss: 2.8088109237153818
Average BLEU-4 Score: 0.02175056287448414
Average Cosine Similarity: 0.6141575306891786
Average Flesch-Kincaid Grade Level Difference: 9.645524296675191


local.py function testing

In [ ]:
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer

def main_desc(functions):
    text = []
    path_to_model = '/content/drive/MyDrive/9900training/desc_model.bin'
    model = AutoModelForSeq2SeqLM.from_pretrained('Salesforce/codet5-base-multi-sum')
    model.load_state_dict(torch.load(path_to_model))
    tokenizer = AutoTokenizer.from_pretrained('Salesforce/codet5-base-multi-sum')

    for function in functions:
        inputs = tokenizer(function, return_tensors="pt", max_length=512, truncation=True)
        summary_ids = model.generate(inputs['input_ids'], max_length=50, num_beams=4, early_stopping=True)
        text.append(tokenizer.decode(summary_ids[0], skip_special_tokens=True))
        print(text)

    return {"description": text}

code = ["""
SSLErrorInfo SSLErrorInfo::CreateError(ErrorType error_type,
                                       net::X509Certificate* cert,
                                       const GURL& request_url) {
  string16 title, details, short_description;
  std::vector<string16> extra_info;
  switch (error_type) {
    case CERT_COMMON_NAME_INVALID: {
      title =
          l10n_util::GetStringUTF16(IDS_CERT_ERROR_COMMON_NAME_INVALID_TITLE);
      std::vector<std::string> dns_names;
      cert->GetDNSNames(&dns_names);
      DCHECK(!dns_names.empty());
      size_t i = 0;
      for (; i < dns_names.size(); ++i) {
        if (dns_names[i] == cert->subject().common_name)
          break;
      }
      if (i == dns_names.size())
        i = 0;
       details =
           l10n_util::GetStringFUTF16(IDS_CERT_ERROR_COMMON_NAME_INVALID_DETAILS,
                                      UTF8ToUTF16(request_url.host()),
                                     UTF8ToUTF16(dns_names[i]),
                                      UTF8ToUTF16(request_url.host()));
       short_description = l10n_util::GetStringUTF16(
           IDS_CERT_ERROR_COMMON_NAME_INVALID_DESCRIPTION);
      extra_info.push_back(
          l10n_util::GetStringUTF16(IDS_CERT_ERROR_EXTRA_INFO_1));
       extra_info.push_back(
           l10n_util::GetStringFUTF16(
               IDS_CERT_ERROR_COMMON_NAME_INVALID_EXTRA_INFO_2,
              UTF8ToUTF16(cert->subject().common_name),
               UTF8ToUTF16(request_url.host())));
       break;
     }
    case CERT_DATE_INVALID:
      extra_info.push_back(
          l10n_util::GetStringUTF16(IDS_CERT_ERROR_EXTRA_INFO_1));
      if (cert->HasExpired()) {
        title = l10n_util::GetStringUTF16(IDS_CERT_ERROR_EXPIRED_TITLE);
        details = l10n_util::GetStringFUTF16(
            IDS_CERT_ERROR_EXPIRED_DETAILS,
            UTF8ToUTF16(request_url.host()),
            UTF8ToUTF16(request_url.host()),
            base::TimeFormatFriendlyDateAndTime(base::Time::Now()));
        short_description =
            l10n_util::GetStringUTF16(IDS_CERT_ERROR_EXPIRED_DESCRIPTION);
        extra_info.push_back(l10n_util::GetStringUTF16(
            IDS_CERT_ERROR_EXPIRED_DETAILS_EXTRA_INFO_2));
      } else {
        title = l10n_util::GetStringUTF16(IDS_CERT_ERROR_NOT_YET_VALID_TITLE);
        details = l10n_util::GetStringFUTF16(
            IDS_CERT_ERROR_NOT_YET_VALID_DETAILS,
            UTF8ToUTF16(request_url.host()),
            UTF8ToUTF16(request_url.host()),
            base::TimeFormatFriendlyDateAndTime(base::Time::Now()));
        short_description =
            l10n_util::GetStringUTF16(IDS_CERT_ERROR_NOT_YET_VALID_DESCRIPTION);
        extra_info.push_back(
            l10n_util::GetStringUTF16(
                IDS_CERT_ERROR_NOT_YET_VALID_DETAILS_EXTRA_INFO_2));
      }
      break;
    case CERT_AUTHORITY_INVALID:
      title = l10n_util::GetStringUTF16(IDS_CERT_ERROR_AUTHORITY_INVALID_TITLE);
      details = l10n_util::GetStringFUTF16(
          IDS_CERT_ERROR_AUTHORITY_INVALID_DETAILS,
          UTF8ToUTF16(request_url.host()));
      short_description = l10n_util::GetStringUTF16(
          IDS_CERT_ERROR_AUTHORITY_INVALID_DESCRIPTION);
      extra_info.push_back(
          l10n_util::GetStringUTF16(IDS_CERT_ERROR_EXTRA_INFO_1));
      extra_info.push_back(l10n_util::GetStringFUTF16(
          IDS_CERT_ERROR_AUTHORITY_INVALID_EXTRA_INFO_2,
          UTF8ToUTF16(request_url.host()),
          UTF8ToUTF16(request_url.host())));
      extra_info.push_back(l10n_util::GetStringUTF16(
          IDS_CERT_ERROR_AUTHORITY_INVALID_EXTRA_INFO_3));
      break;
    case CERT_CONTAINS_ERRORS:
      title = l10n_util::GetStringUTF16(IDS_CERT_ERROR_CONTAINS_ERRORS_TITLE);
      details = l10n_util::GetStringFUTF16(
          IDS_CERT_ERROR_CONTAINS_ERRORS_DETAILS,
          UTF8ToUTF16(request_url.host()));
      short_description =
          l10n_util::GetStringUTF16(IDS_CERT_ERROR_CONTAINS_ERRORS_DESCRIPTION);
      extra_info.push_back(
          l10n_util::GetStringFUTF16(IDS_CERT_ERROR_EXTRA_INFO_1,
                                     UTF8ToUTF16(request_url.host())));
      extra_info.push_back(l10n_util::GetStringUTF16(
          IDS_CERT_ERROR_CONTAINS_ERRORS_EXTRA_INFO_2));
      break;
    case CERT_NO_REVOCATION_MECHANISM:
      title = l10n_util::GetStringUTF16(
          IDS_CERT_ERROR_NO_REVOCATION_MECHANISM_TITLE);
      details = l10n_util::GetStringUTF16(
          IDS_CERT_ERROR_NO_REVOCATION_MECHANISM_DETAILS);
      short_description = l10n_util::GetStringUTF16(
          IDS_CERT_ERROR_NO_REVOCATION_MECHANISM_DESCRIPTION);
      break;
    case CERT_UNABLE_TO_CHECK_REVOCATION:
      title = l10n_util::GetStringUTF16(
          IDS_CERT_ERROR_UNABLE_TO_CHECK_REVOCATION_TITLE);
      details = l10n_util::GetStringUTF16(
          IDS_CERT_ERROR_UNABLE_TO_CHECK_REVOCATION_DETAILS);
      short_description = l10n_util::GetStringUTF16(
          IDS_CERT_ERROR_UNABLE_TO_CHECK_REVOCATION_DESCRIPTION);
      break;
    case CERT_REVOKED:
      title = l10n_util::GetStringUTF16(IDS_CERT_ERROR_REVOKED_CERT_TITLE);
      details = l10n_util::GetStringFUTF16(IDS_CERT_ERROR_REVOKED_CERT_DETAILS,
                                           UTF8ToUTF16(request_url.host()));
      short_description =
          l10n_util::GetStringUTF16(IDS_CERT_ERROR_REVOKED_CERT_DESCRIPTION);
      extra_info.push_back(
          l10n_util::GetStringUTF16(IDS_CERT_ERROR_EXTRA_INFO_1));
      extra_info.push_back(
          l10n_util::GetStringUTF16(IDS_CERT_ERROR_REVOKED_CERT_EXTRA_INFO_2));
      break;
    case CERT_INVALID:
      title = l10n_util::GetStringUTF16(IDS_CERT_ERROR_INVALID_CERT_TITLE);
      details = l10n_util::GetStringFUTF16(
          IDS_CERT_ERROR_INVALID_CERT_DETAILS,
          UTF8ToUTF16(request_url.host()));
      short_description =
          l10n_util::GetStringUTF16(IDS_CERT_ERROR_INVALID_CERT_DESCRIPTION);
      extra_info.push_back(
          l10n_util::GetStringUTF16(IDS_CERT_ERROR_EXTRA_INFO_1));
      extra_info.push_back(l10n_util::GetStringUTF16(
          IDS_CERT_ERROR_INVALID_CERT_EXTRA_INFO_2));
      break;
    case CERT_WEAK_SIGNATURE_ALGORITHM:
      title = l10n_util::GetStringUTF16(
          IDS_CERT_ERROR_WEAK_SIGNATURE_ALGORITHM_TITLE);
      details = l10n_util::GetStringFUTF16(
          IDS_CERT_ERROR_WEAK_SIGNATURE_ALGORITHM_DETAILS,
          UTF8ToUTF16(request_url.host()));
      short_description = l10n_util::GetStringUTF16(
          IDS_CERT_ERROR_WEAK_SIGNATURE_ALGORITHM_DESCRIPTION);
      extra_info.push_back(
          l10n_util::GetStringUTF16(IDS_CERT_ERROR_EXTRA_INFO_1));
      extra_info.push_back(
          l10n_util::GetStringUTF16(
              IDS_CERT_ERROR_WEAK_SIGNATURE_ALGORITHM_EXTRA_INFO_2));
      break;
    case CERT_WEAK_KEY:
      title = l10n_util::GetStringUTF16(IDS_CERT_ERROR_WEAK_KEY_TITLE);
      details = l10n_util::GetStringFUTF16(
          IDS_CERT_ERROR_WEAK_KEY_DETAILS, UTF8ToUTF16(request_url.host()));
      short_description = l10n_util::GetStringUTF16(
          IDS_CERT_ERROR_WEAK_KEY_DESCRIPTION);
      extra_info.push_back(
          l10n_util::GetStringUTF16(IDS_CERT_ERROR_EXTRA_INFO_1));
      extra_info.push_back(
          l10n_util::GetStringUTF16(
              IDS_CERT_ERROR_WEAK_KEY_EXTRA_INFO_2));
      break;
    case UNKNOWN:
      title = l10n_util::GetStringUTF16(IDS_CERT_ERROR_UNKNOWN_ERROR_TITLE);
      details = l10n_util::GetStringUTF16(IDS_CERT_ERROR_UNKNOWN_ERROR_DETAILS);
      short_description =
          l10n_util::GetStringUTF16(IDS_CERT_ERROR_UNKNOWN_ERROR_DESCRIPTION);
      break;
    default:
      NOTREACHED();
  }
  return SSLErrorInfo(title, details, short_description, extra_info);
}"""]
description = main_desc(code)

['The vulnerability in the `CreateError` function was caused by the lack of proper validation for the error_type parameter in the SSLErrorInfo::CreateError function. Specifically, if the error_type parameter was CERT_COMMON']


In [ ]:
import pandas as pd
import numpy as np
from transformers import RobertaTokenizer, T5ForConditionalGeneration, T5Config, AutoModelForSeq2SeqLM, AutoTokenizer, get_linear_schedule_with_warmup
from torch.utils.data import DataLoader, Dataset, RandomSampler, SequentialSampler
import torch
import torch.optim as optim
from tqdm import tqdm
import logging
import os
import spacy


logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

class Args:
    def __init__(self):
        self.train_data_file = '/content/drive/MyDrive/9900training/train.csv'
        self.eval_data_file = '/content/drive/MyDrive/9900training/val.csv'
        self.test_data_file = '/content/drive/MyDrive/9900training/test.csv'
        self.block_size = 512
        self.train_batch_size = 8
        self.val_batch_size = 8
        self.epochs = 10
        # # quick test
        self.sample_size = 50

args = Args()

model = AutoModelForSeq2SeqLM.from_pretrained('Salesforce/codet5-base-multi-sum')
tokenizer = AutoTokenizer.from_pretrained('Salesforce/codet5-base-multi-sum')

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)


class InputFeatures(object):
    def __init__(self, input_tokens, input_ids, descs):
        self.input_tokens = input_tokens
        self.input_ids = input_ids
        self.descs = descs

def convert_examples_to_features(func, desc, tokenizer, args):
    source_tokens = tokenizer.tokenize(str(func))[:args.block_size-2]
    # deleted that part because we dont know the details of t5 tokenizer, that
    # is particularlly for codebert
    source_ids = tokenizer.convert_tokens_to_ids(source_tokens)
    padding_length = args.block_size - len(source_ids)
    source_ids += [tokenizer.pad_token_id] * padding_length
    return InputFeatures(source_tokens, source_ids, str(desc))

class CodeToTextDataset(Dataset):
    def __init__(self, tokenizer, args, file_type):
        if file_type == "train":
            file_path = args.train_data_file
        elif file_type == "eval":
            file_path = args.eval_data_file
        elif file_type == "test":
            file_path = args.test_data_file

        self.examples = []
        # df = pd.read_csv(file_path)
        # quick testing
        df = pd.read_csv(file_path).sample(n=args.sample_size)
        funcs = df["vulnerable_code"].tolist()
        descs = df["Explanation_of_Vulnerability_In_Context"].tolist()
        for i in tqdm(range(len(funcs)), desc="Loading data"):
            self.examples.append(convert_examples_to_features(funcs[i],descs[i], tokenizer, args))
        if file_type == "train":
            for example in self.examples[:3]:
                logger.info("*** Example ***")
                logger.info("desc: {}".format(example.descs))
                logger.info("input_tokens: {}".format([x.replace('\u0120','_') for x in example.input_tokens]))
                logger.info("input_ids: {}".format(' '.join(map(str, example.input_ids))))

    def __len__(self):
        return len(self.examples)

    def __getitem__(self, idx):
        example = self.examples[idx]
        return {
            'input_ids': torch.tensor(example.input_ids, dtype=torch.long),
            'desc': torch.tensor(tokenizer.encode(example.descs, padding='max_length', truncation=True, max_length=args.block_size), dtype=torch.long),
        }

def train(train_dataset, model, args, tokenizer):
    train_sampler = RandomSampler(train_dataset)
    train_dataloader = DataLoader(train_dataset, sampler=train_sampler, batch_size=args.train_batch_size, num_workers=0)

    optimizer = optim.AdamW(model.parameters(), lr=5e-5)
    total_steps = len(train_dataloader) * args.epochs
    scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=0, num_training_steps=total_steps)

    for epoch in range(args.epochs):
        model.train()
        for batch in train_dataloader:
            input_ids = batch['input_ids'].to(device)
            descs = batch['desc'].to(device)

            outputs = model(input_ids=input_ids, labels=descs)
            loss = outputs.loss

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            scheduler.step()

            print(f'Epoch: {epoch}, Loss: {loss.item()}')
    model_to_save = model.module if hasattr(model,'module') else model
    torch.save(model_to_save.state_dict(), '/content/drive/MyDrive/9900training/custom_train_t5.bin')


def test(test_dataset, model, tokenizer, args):
    test_sampler = SequentialSampler(test_dataset)
    test_dataloader = DataLoader(test_dataset, sampler=test_sampler, batch_size=1, num_workers=0)

    model.eval()
    total_loss = 0
    for batch in tqdm(test_dataloader, desc="Testing"):
        input_ids = batch['input_ids'].to(device)
        true_desc = batch['desc'].to(device)

        with torch.no_grad():
            outputs = model.generate(input_ids=input_ids, max_length=args.block_size)

        predicted_desc = tokenizer.decode(outputs[0], skip_special_tokens=True)
        true_desc_decoded = tokenizer.decode(true_desc[0], skip_special_tokens=True)

        logger.info(f"True Description: {true_desc_decoded}")
        logger.info(f"Predicted Description: {predicted_desc}")
        print(f"True Description: {true_desc_decoded}")
        print(f"Predicted Description: {predicted_desc}")


def test_model(test_dataset, tokenizer, args):
    path_to_model = '/content/drive/MyDrive/9900training/desc_model.bin'
    model.load_state_dict(torch.load(path_to_model))
    test_sampler = SequentialSampler(test_dataset)
    test_dataloader = DataLoader(test_dataset, sampler=test_sampler, batch_size=1, num_workers=0)

    model.eval()
    total_loss = 0
    for batch in tqdm(test_dataloader, desc="Testing"):
        input_ids = batch['input_ids'].to(device)
        true_desc = batch['desc'].to(device)

        with torch.no_grad():
            outputs = model.generate(input_ids=input_ids, max_length=args.block_size)

        predicted_desc = tokenizer.decode(outputs[0], skip_special_tokens=True)
        true_desc_decoded = tokenizer.decode(true_desc[0], skip_special_tokens=True)

        logger.info(f"True Description: {true_desc_decoded}")
        logger.info(f"Predicted Description: {predicted_desc}")
        print(f"True Description: {true_desc_decoded}")
        print(f"Predicted Description: {predicted_desc}")

def evaluate(val_dataset, model, args, tokenizer):
    val_sampler = SequentialSampler(val_dataset)
    val_dataloader = DataLoader(val_dataset, sampler=val_sampler, batch_size=args.val_batch_size, num_workers=0)

    model.eval()
    total_loss = 0
    for batch in val_dataloader:
        input_ids = batch['input_ids'].to(device)
        descs = batch['desc'].to(device)

        with torch.no_grad():
            outputs = model(input_ids=input_ids, labels=descs)
            loss = outputs.loss
            total_loss += loss.item()

    print(f'Validation Loss: {total_loss / len(val_dataloader)}')

def generate_text(model, tokenizer, code, max_len=512):
    model.eval()
    input_enc = tokenizer(code, max_length=max_len, padding='max_length', truncation=True, return_tensors="pt")
    input_ids = input_enc['input_ids'].to(device)

    with torch.no_grad():
        outputs = model.generate(input_ids=input_ids, max_length=max_len)

    return tokenizer.decode(outputs[0], skip_special_tokens=True)

# train_dataset = CodeToTextDataset(tokenizer, args, 'train')
# train(train_dataset, model, args, tokenizer)

# test_dataset = CodeToTextDataset(tokenizer, args, 'test')
# test(test_dataset, model, tokenizer, args)

# val_dataset = CodeToTextDataset(tokenizer, args, 'eval')
# evaluate(val_dataset, model, args, tokenizer)

test_dataset = CodeToTextDataset(tokenizer, args, 'test')
test_model(test_dataset, tokenizer, args)

Testing:   2%|▏         | 1/50 [00:03<03:11,  3.90s/it]

True Description: The vulnerability in the code was caused by passing the address of the `append` structure incorrectly to the `FreeStmt` function, potentially leading to an invalid free operation. This incorrect passing of the address could result in attempting to free memory that has already been deallocated or is not valid, causing a use-after-free scenario. In the vulnerable state, if a local attacker supplies a crafted keymap file, it could crash the keymap parsers in xkbcommon or potentially exploit the memory corruption for other unspecified impacts, highlighting the critical importance of proper memory management practices to prevent such vulnerabilities.
Predicted Description: The vulnerability in the code was caused by the lack of proper initialization for the `keysym_list` variable in the `expr` variable before appending the keysym list to the `append` variable. This oversight allowed an attacker to manipulate the `keysym_list` variable without ensuring that the `keysym_list

Testing:   4%|▍         | 2/50 [00:06<02:25,  3.03s/it]

True Description: The vulnerability in the code was caused by mishandling reference counts associated with the `f2fs_wait_discard_bios` calls in the Linux kernel implementation prior to version 4.14. This mishandling led to a scenario where local users could trigger a denial of service by exploiting the reference count issue, particularly in the context of the `fstrim` command. The incorrect management of reference counts within the `f2fs_wait_discard_bios` function allowed for a bug to be triggered, impacting the system's stability and potentially leading to a denial of service situation.
Predicted Description: The vulnerability in the `f2fs_wait_discard_bios` function was caused by a lack of proper validation for the `discard_bios` flag in the `f2fs_wait_discard_bios` function. When the `discard_bios` flag was set to false, the function would attempt to read the `discard_bios` command without verifying the existence of the `discard_bios` flag. This oversight allowed for the possibili

Testing:   6%|▌         | 3/50 [00:08<02:11,  2.79s/it]

...

True Description: The cause of the vulnerability was the improper handling of incoming packets, specifically the DCCP_PKT_REQUEST type, in the `dccp_rcv_state_process` function when the socket was in the LISTEN state. The original code did not consume the packet after processing a connection request, which led to a situation where the same packet could be processed multiple times, resulting in a double free condition. This flaw allowed attackers to exploit the mishandling of packet data structures, potentially leading to privilege escalation or denial of service, particularly when using the IPV6_RECVPKTINFO option with the setsockopt system call.
Predicted Description: The vulnerability in the `dccp_rcv_state_process` function was caused by the lack of proper validation for the packet type before processing it. Specifically, the original code did not adequately check for the packet type before processing it, leading to a NULL pointer dereference and potential potential denial of servic

Testing:  98%|█████████▊| 49/50 [01:47<00:02,  2.06s/it]

True Description: The vulnerability in the code was caused by improper validation of packet lengths in the `handle_rx` and `get_rx_bufs` functions in the `drivers/vhost/net.c` file of the Linux kernel. Specifically, when mergeable buffers were disabled, the code did not adequately verify the lengths of incoming packets. This lack of validation allowed guest OS users to send maliciously crafted packets with incorrect lengths, leading to memory corruption and potentially triggering a denial of service (DoS) condition or privilege escalation on the host OS. Proper validation of packet lengths is crucial to prevent such security vulnerabilities and ensure the integrity of packet processing mechanisms.
Predicted Description: The vulnerability in the `handle_rx` function was caused by a lack of proper validation for the `total_len` variable before attempting to read or write data from the `nvq->vq` structure. This oversight allowed local users to trigger a denial of service by sending a craf

Testing: 100%|██████████| 50/50 [01:49<00:00,  2.20s/it]

True Description: The cause of the vulnerability was the reliance on the `RAND_pseudo_bytes` function, which is known to produce pseudo-random bytes that may not meet the necessary cryptographic strength required for secure applications. This function can generate predictable outputs, making it easier for attackers to exploit weaknesses in cryptographic protection mechanisms. In the original implementation of the `openssl_random_pseudo_bytes` function, this reliance compromised the security of the generated random bytes, allowing potential remote attacks to defeat the intended cryptographic safeguards. The updated code mitigates this risk by using a more secure alternative for random byte generation.
Predicted Description: The vulnerability in the PHP code was caused by the lack of proper validation for the `strong_result` variable in the `openssl_random_pseudo_bytes` function. This vulnerability allowed remote attackers to exploit the vulnerability by providing a crafted application w

In [5]:
!pip install nltk sklearn textstat spacy

  error: subprocess-exited-with-error
  
  × python setup.py egg_info did not run successfully.
  │ exit code: 1
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  Preparing metadata (setup.py) ... error
error: metadata-generation-failed

× Encountered error while generating package metadata.
╰─> See above for output.

note: This is an issue with the package mentioned above, not pip.
hint: See above for details.


In [4]:
!python -m spacy download en_core_web_sm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 104.7 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


In [3]:
!pip install textstat

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.1/105.1 kB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 54.8 MB/s eta 0:00:00
